In [1]:
import numpy as np
import scipy.optimize as optimize
import time

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_2(x, Ut,Lt,Vt):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_3(x, Ut,Lt,Vt):
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * (np.cos(x[i+Ut.shape[0]])) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_4(x, Ut,Lt,Vt):
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = (np.cos(x[i]) / l0) * (np.cos(x[i+Ut.shape[0]]))
      U_new[i][1] = (np.cos(x[i]) / l1) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.cos(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+4*Ut.shape[0]])
      U_new[i][3] = (np.sin(x[i]) / l3) * (np.sin(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+5*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


""" Cost Function """
def costFn(x, Ut,Lt,Vt,B,k):
    if k==2:
      Bp = newMat_2(x, Ut, Lt, Vt)
    elif k==3:
      Bp = newMat_3(x, Ut, Lt, Vt)
    elif k==4:
      Bp = newMat_4(x, Ut, Lt, Vt)
    else:
      return 0
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

In [3]:
""" Calculate Results """
def calcResults(k, m ,n):
    res = []
    times = []
    print("K = ", k, ", M = ", m, ", N = ", n)
    for i in range(10):
        A = np.random.rand(m, n)
        for j in range(m): A[j] /= sum(A[j])
        B = np.sqrt(A)
        U, L, V = np.linalg.svd(B, full_matrices=False)
        size = (2*m*(k-1))
        initial_guess = np.ones((size,), dtype=np.longdouble)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]
        Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
        bounds = []
        for _ in range(size): bounds.append((-np.pi, np.pi))

        start1 = time.time()
        result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Lt,Vt,B,k),
                                    tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
        time1 = time.time()-start1
        start2 = time.time()
        result2 = optimize.differential_evolution(func=costFn, args=(Ut,Lt,Vt,B,k), bounds=bounds, seed=0, tol=1e-7)
        time2 = time.time()-start2
        
        res.append([np.linalg.norm(B**2 - Bt**2), costFn(result.x,Ut,Lt,Vt,B,k), costFn(result2.x,Ut,Lt,Vt,B,k)])
        times.append([time1, time2])
        print(i, " ", end='')
    print('\n')
    return np.array(res), np.array(times)

In [4]:
def printStatistics(res, times):
    print("Loss in truncation vs Loss after Nelder-Mead vs Loss after Differential Evolution: ")
    print(res)
    print("\nTime taken for Nelder-Mead vs Time taken to complete differential evolution: ")
    print(times)
    print("\nMean Losses: ", res.mean(axis=0))
    print("\nMean Execution Times: ", times.mean(axis=0))

In [5]:
k= 2
printStatistics(*calcResults(k, k+1, k+1))

K =  2 , M =  3 , N =  3
0  1  2  3  4  5  6  7  8  9  

Loss in truncation vs Loss after Nelder-Mead vs Loss after Differential Evolution: 
[[0.10987973 0.07937117 0.07937117]
 [0.16342922 0.12411882 0.12411882]
 [0.11269724 0.07021244 0.07021244]
 [0.00655305 0.00457533 0.00457533]
 [0.38978782 0.35230873 0.35230873]
 [0.10290044 0.09564625 0.09560268]
 [0.20837867 0.13924786 0.13924787]
 [0.07403156 0.03438314 0.03438314]
 [0.03034444 0.02479427 0.02479427]
 [0.07011035 0.04544871 0.04544871]]

Time taken for Nelder-Mead vs Time taken to complete differential evolution: 
[[ 0.61812377  8.42601848]
 [ 0.62053442  4.87711215]
 [ 0.49073052  7.28753495]
 [ 0.22187138  5.32305574]
 [ 0.26727152  3.34424806]
 [ 0.57065034 10.10132623]
 [ 0.15864682  4.45150781]
 [ 0.15596533  4.14570761]
 [ 0.54135251 12.68482852]
 [ 0.18165469  4.40873408]]

Mean Losses:  [0.12681125 0.09701067 0.09700631]

Mean Execution Times:  [0.38268013 6.50500736]


In [6]:
k= 3
printStatistics(*calcResults(k, k+1, k+1))

K =  3 , M =  4 , N =  4
0  1  2  3  4  5  6  7  8  9  

Loss in truncation vs Loss after Nelder-Mead vs Loss after Differential Evolution: 
[[1.09781152e-02 3.83175413e-09 3.95438015e-08]
 [3.43465223e-02 1.57485841e-02 1.95281462e-08]
 [1.20865147e-01 9.06723146e-02 8.80369326e-02]
 [9.57381882e-03 4.84791119e-03 6.95613918e-03]
 [2.20252918e-02 1.68846254e-02 1.68844929e-02]
 [3.26430744e-02 2.38567729e-02 3.23185905e-08]
 [8.40399834e-02 1.00338703e-09 1.88205402e-08]
 [1.42589246e-01 3.36500203e-03 1.49803650e-08]
 [3.35832754e-02 2.77638876e-02 2.46404901e-08]
 [4.49191616e-02 3.43639275e-02 3.21647687e-02]]

Time taken for Nelder-Mead vs Time taken to complete differential evolution: 
[[  3.86894107 150.56200266]
 [  7.09144402 161.66978812]
 [  7.45628333 150.27913046]
 [ 14.00636649 155.8259325 ]
 [ 11.84432006 158.56198382]
 [  6.44989133 159.66674113]
 [  6.46204686 156.99303079]
 [  7.39602804 158.04089403]
 [  5.02198792 157.36519432]
 [ 10.16790724 150.84926152]]

Mean Lo

In [7]:
k= 4
printStatistics(*calcResults(k, k+1, k+1))

K =  4 , M =  5 , N =  5
0  1  2  3  4  5  6  7  8  9  

Loss in truncation vs Loss after Nelder-Mead vs Loss after Differential Evolution: 
[[3.35303600e-02 3.84582879e-02 5.36842852e-02]
 [7.22014651e-03 3.87252894e-02 4.84426724e-03]
 [1.87994621e-02 1.45804634e-09 1.24995131e-07]
 [1.83041210e-02 1.44404833e-08 2.38608665e-01]
 [2.07602510e-02 9.89338310e-10 2.94827571e-02]
 [3.53459182e-02 6.31714362e-04 1.52107445e-02]
 [1.26158587e-01 4.33017242e-02 2.25875913e-02]
 [9.66563268e-02 7.08200960e-03 6.16314889e-08]
 [9.02452546e-02 5.46310294e-02 9.36368781e-02]
 [2.02110004e-02 3.86695005e-03 3.27808044e-06]]

Time taken for Nelder-Mead vs Time taken to complete differential evolution: 
[[ 38.45283318 440.73033357]
 [ 43.56033611 425.48212552]
 [ 21.24414945 484.48989391]
 [ 23.38868999 489.40111017]
 [ 35.58993793 449.60278964]
 [ 66.31484675 433.98526525]
 [ 36.27268028 437.0058167 ]
 [ 43.17346644 449.72630405]
 [ 41.95279717 443.08637452]
 [ 46.67003846 411.64376855]]

Mean Lo